In [1]:
# import csv

# csv_file_name = 'JMIS_data.csv'

# columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords"]
# with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=columns)
#     writer.writeheader()

# def add_row_to_csv(data):
#     with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
#         writer = csv.DictWriter(file, fieldnames=columns)
#         writer.writerow(data)

In [2]:
import csv

csv_file_name = 'JMIS_data.csv'

columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords"]

def add_row_to_csv(data):
    with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writerow(data)

In [3]:
def get_credentials():
    with open('../credentials.txt', 'r') as file:
        lines = file.readlines()
        username = lines[0].strip()
        password = lines[1].strip()
        file.close()
        return username, password


In [4]:
def overwrite_line(line_number, new_value):
    with open('progress.txt', 'r') as file:
        lines = file.readlines()
        
    if 0 <= line_number < len(lines):
        lines[line_number] = f"{new_value}\n" 
        with open('progress.txt', 'w') as file:
            file.writelines(lines)
        return True
    else:
        return False

def read_numbers():
    with open('progress.txt', 'r') as file:
        numbers = [int(line.strip()) for line in file]
    return numbers[0], numbers[1], numbers[2]

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
import urllib
import os
import shutil
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Initial_path = r"C:\Users\yitong\Desktop\Research-Assistant\Scrape\JMIS\JMIS_PDF"

In [6]:
def get_one_article(driver, wait):

    # Function to find the following dd for a given dt text
    def find_following_dd_by_dt_text(dt_text):
        dt_elements = driver.find_elements(By.XPATH, "//dl[@id='citationFields']/dt")
        for dt in dt_elements:
            if dt.text.strip(':') == dt_text:
                return dt.find_element(By.XPATH, "./following-sibling::dd[1]")
        return None

    title = None
    authors = None
    keywords_string = None
    publish_date = None
    abstract_text = None

    # Extract Title
    try:
        title_element = driver.find_element(By.CLASS_NAME, "NLM_article-title")
        title = title_element.text
    except:
        pass

    # Extract Authors
    try:
        author_elements = driver.find_elements(By.CLASS_NAME, 'author')
        authors = []
        for author in author_elements:
            authors.append(author.text)
        authors = "; ".join(authors)
    except:
        pass

    # Extract Keywords
    try:
        keyword_elements = driver.find_elements(By.CSS_SELECTOR, ".abstractKeywords .kwd-btn")
        keywords = [keyword.text for keyword in keyword_elements]
        keywords_string = "; ".join(keywords)

    except:
        pass

    # Extract Publish Date (assuming it's within the Source information)
    try:
        publish_date_element = driver.find_element(By.XPATH, "//div[contains(@class, 'itemPageRangeHistory')]//span[contains(text(), 'Published online')]")
        publish_date = publish_date_element.text
    except:
        pass


    try:
        # Locate the <div> that contains the abstract and then get the text from the <p> tag inside it
        abstract_element = driver.find_element(By.CSS_SELECTOR, ".hlFld-Abstract p.last")
        abstract_text = abstract_element.text

    except:
        pass

    # print(f"Title: {title}")
    # print(f"Authors: {authors}")
    # print(f"Publish Date: {publish_date}")
    # print(f"Abstract: {abstract_text}")
    # print(f"Keywords: {keywords_string}")


    add_row_to_csv({
        "Title": title,
        "Authors": authors,
        "Publish Date": publish_date,
        "Abstract": abstract_text,
        "Keywords": keywords_string,
    })

    try:
        pdf_link_element = driver.find_element("link text", 'Download PDF')
        pdf_link_element.click()

        time.sleep(10)
        result_list_element = driver.find_element(By.XPATH, "//a[@class='resultListLink link color-s1']")
        result_list_element.click()
    except NoSuchElementException:
        time.sleep(5)
        pass

    window_handles = driver.window_handles
    driver.close()
    driver.switch_to.window(window_handles[1])
    
    time.sleep(5)


In [7]:
def recover_from_breakpoint(year, issue, article):
    
    user_id, password = get_credentials()

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": Initial_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True 
    })

    wd = webdriver.Chrome(service = Service(r'../chromedriver.exe'), options=options)
    wait = WebDriverWait(wd, 60)
    
    wd.get("https://www-tandfonline-com.libproxy1.nus.edu.sg/loi/mmis20")
    button = wd.find_element(By.ID, 'btn_nus')
    button.click()
    wait.until(EC.element_to_be_clickable((By.ID, 'userNameInput')))
    user_id_box = wd.find_element(By.ID, 'userNameInput')
    password_box = wd.find_element(By.ID, 'passwordInput')
    user_id_box.send_keys(user_id)
    password_box.send_keys(password)
    submit_button = wd.find_element(By.ID, 'submitButton')
    submit_button.click()
    
    wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler')))
    cookie_button_element = wd.find_element(By.ID, "onetrust-accept-btn-handler")
    cookie_button_element.click()

    time.sleep(5)
    volume_elements = wd.find_elements(By.XPATH, "//li[contains(@class, 'vol_li')]")

    for element in volume_elements:
        # Example action: Print the text of the button within each 'li' element
        volume_button = element.find_element(By.CLASS_NAME, 'volume_link').text
    volume_elements_len = len(volume_elements)


    # for element in vol_li_elements:
    #     # Find the button within this 'vol_li' element
    #     volume_link = element.find_element(By.CLASS_NAME, 'volume_link')
        
    #     # Accessing the button's attributes
    #     aria_expanded = volume_link.get_attribute('aria-expanded')
    #     data_id = volume_link.get_attribute('data-id')
    #     vol_id = volume_link.get_attribute('id')
    #     volume_text = volume_link.text  # The text of the button
        
    #     # Optionally, you could also find the <a> tag if needed
    #     hidden_link = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
        
    for j in range(year, volume_elements_len):
        overwrite_line(0, j)
        volume_elements[j].click()
        issue_class = "issue-link"
        wait.until(EC.element_to_be_clickable((By.CLASS_NAME, issue_class)))
        issue_elements = wd.find_elements(By.CLASS_NAME, issue_class)
        issue_elements_len = len(issue_elements)
        # print(issue_elements_len)
        for i in range(issue, issue_elements_len):
            # article = 0
            # overwrite_line(2, 0)
            overwrite_line(1, i)
            # issue_elements[i].click()
            ActionChains(wd).move_to_element(issue_elements[i]).key_down(Keys.CONTROL).click(issue_elements[i]).key_up(Keys.COMMAND).perform()
            time.sleep(2)
            window_handles = wd.window_handles
            wd.switch_to.window(window_handles[1])

            # markall_class = 'checkbox-container'
            # wait.until(EC.element_to_be_clickable((By.CLASS_NAME, markall_class)))
            # markall_element = wd.find_element(By.CLASS_NAME, markall_class)
            # markall_element.click()

            # pdf_icon_link = wd.find_element(By.XPATH, '//a[./*[@class="fa fa-file-pdf-o"]]')
            # pdf_icon_link.click()

            # # start_download_button_id = 'btn-download-pdfs'
            # # wait.until(EC.element_to_be_clickable((By.ID, start_download_button_id)))
            # # start_download_button = wd.find_element(By.ID, start_download_button_id)
            # # start_download_button.click()

            # button_text = "Yes, download zip file"
            # wait.until(EC.element_to_be_clickable((By.XPATH, f"//button[text()='{button_text}']")))
            # button = wd.find_element(By.XPATH, f"//button[text()='{button_text}']")
            # button.click()
            # time.sleep(60)

            title_elements = wd.find_elements(By.CLASS_NAME, "hlFld-Title")
            # print('title_elements', len(title_elements))
            for k in range(article, len(title_elements)):
                overwrite_line(2, k)
                # title_elements[k].click()
                ActionChains(wd).move_to_element(title_elements[k]).key_down(Keys.CONTROL).click(title_elements[k]).key_up(Keys.COMMAND).perform()
                time.sleep(2)
                window_handles = wd.window_handles
                wd.switch_to.window(window_handles[2])
                print(j, i, k)
                time.sleep(5)
                get_one_article(wd, wait)  
            article = 0 
            overwrite_line(2, 0)
            window_handles = wd.window_handles
            wd.close()
            wd.switch_to.window(window_handles[0])
            time.sleep(2)
            # volume_elements = wd.find_elements(By.XPATH, "//li[contains(@class, 'vol_li')]")
            # volume_elements[j].click()
            issue_class = "issue-link"
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME, issue_class)))
            issue_elements = wd.find_elements(By.CLASS_NAME, issue_class)
            print("finish issue")

        issue = 0
        overwrite_line(1, 0)
        print("finish year")
        # overwrite_line(0, j + 1)


In [8]:
while True:
    try:
        year, issue, article = read_numbers()
        recover_from_breakpoint(year, issue, article)
    except Exception as e:
        print(e)
        print("Failed")

23 3 2
23 3 3
23 3 4
23 3 5
23 3 6
23 3 7
23 3 8
finish issue
finish year
Message: 

Failed
24 0 0
24 0 1
24 0 2
24 0 3
24 0 4
24 0 5
24 0 6
24 0 7
24 0 8
24 0 9
finish issue
24 1 0
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 1 6
24 1 7
24 1 8
finish issue
list index out of range
Failed
24 2 0
24 2 1
24 2 2
24 2 3
24 2 4
24 2 5
24 2 6
24 2 7
24 2 8
24 2 9
finish issue
list index out of range
Failed
24 3 0
24 3 1
24 3 2
24 3 3
24 3 4
24 3 5
24 3 6
24 3 7
24 3 8
24 3 9
finish issue
finish year
Message: 

Failed
25 0 0
25 0 1
25 0 2
25 0 3
25 0 4
25 0 5
25 0 6
25 0 7
25 0 8
25 0 9
25 0 10
finish issue
25 1 0
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 1 9
25 1 10
finish issue
25 2 0
25 2 1
25 2 2
25 2 3
25 2 4
25 2 5
25 2 6
25 2 7
25 2 8
25 2 9
25 2 10
finish issue
25 3 0
25 3 1
25 3 2
25 3 3
25 3 4
25 3 5
25 3 6
25 3 7
25 3 8
25 3 9
25 3 10
finish issue
finish year
Message: 

Failed
26 0 0
26 0 1
26 0 2
26 0 3
26 0 4
26 0 5
26 0 6
26 0 7
26 0 8
26 0 9
26 0 10
finish issue
26 1 0